# Getting Started

In [1]:
# # Upload Kaggle json

# !pip install -q kaggle
# !pip install -q kaggle-cli
# !mkdir -p ~/.kaggle
# !cp "/content/drive/My Drive/Kaggle/kaggle.json" ~/.kaggle/ # Mount GDrive
# !cat ~/.kaggle/kaggle.json 
# !chmod 600 ~/.kaggle/kaggle.json
# !kaggle competitions download -c fake-news -p dataset
# !unzip /content/dataset/train.csv.zip
# !unzip /content/dataset/test.csv.zip

In [4]:
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
from matplotlib import rcParams
plt.rcParams['figure.figsize'] = [10,10]
import seaborn as sns
sns.set_theme(style="darkgrid")
from wordcloud import WordCloud

import nltk
from nltk import sent_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')
from nltk.tokenize import word_tokenize

import contractions

import itertools
import datetime
import time
from collections import Counter
import string

import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nroy0\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\nroy0\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nroy0\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


  Using cached contractions-0.0.48-py2.py3-none-any.whl (6.4 kB)
  Using cached textsearch-0.0.21-py2.py3-none-any.whl (7.5 kB)
  Using cached pyahocorasick-1.4.2.tar.gz (321 kB)
  Running setup.py clean for pyahocorasick
Failed to build pyahocorasick
    Running setup.py install for pyahocorasick: started
    Running setup.py install for pyahocorasick: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\nroy0\anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\nroy0\\AppData\\Local\\Temp\\pip-install-hgawtbmu\\pyahocorasick\\setup.py'"'"'; __file__='"'"'C:\\Users\\nroy0\\AppData\\Local\\Temp\\pip-install-hgawtbmu\\pyahocorasick\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\nroy0\AppData\Local\Temp\pip-wheel-65rqdycn'
       cwd: C:\Users\nroy0\AppData\Local\Temp\pip-install-hgawtbmu\pyahocorasick\
  Complete output (5 lines):
  running bdist_wheel
  running build
  running build_ext
  building 'ahocorasick' extension
  error: Microsoft Visual C++ 14.0 is required. Get it with "Build Tools for Visual Studio": https://visualstudio.microsoft.com/downloads/
  ----------------------------------------
  ERROR: Failed building wh

ModuleNotFoundError: No module named 'contractions'

In [3]:
# train_df = pd.read_csv('/content/train.csv', header=0)
# test_df = pd.read_csv('/content/test.csv', header=0)

# train_df = pd.read_csv('/content/train.csv', header=0)
# test_df = pd.read_csv('/content/test.csv', header=0)

In [4]:
train_df = train_df.fillna(' ')
test_df = test_df.fillna(' ')

train_df['text'] = train_df['text'].str.strip()
test_df['text'] = test_df['text'].str.strip()

train_df['raw_text_length'] = train_df['text'].apply(lambda x: len(x))
print(len(train_df[train_df['raw_text_length']==0]))

116


In [5]:
train_df = train_df[train_df['raw_text_length'] > 0]
train_df.shape

(20684, 6)

In [6]:
def preprocess_text(x):
  try:
    cleaned_text = re.sub(r'[^a-zA-Z\d\s\']+', '', x)
    word_list = []
    for each_word in cleaned_text.split(' '):
        word_list.append(contractions.fix(each_word).lower())
  except:
    print(x)
  return " ".join(word_list)

In [7]:
text_cols = ['text', 'title', 'author']

In [8]:
%%time
for col in text_cols:
  print("Processing column: {}".format(col))
  print("Processing train data... ")
  train_df[col] = train_df[col].apply(lambda x: preprocess_text(x))
  print("Processing test data... ")
  test_df[col] = test_df[col].apply(lambda x: preprocess_text(x)) 

Processing column: text
Processing train data... 
Processing test data... 
Processing column: title
Processing train data... 
Processing test data... 
Processing column: author
Processing train data... 
Processing test data... 
CPU times: user 1min, sys: 222 ms, total: 1min
Wall time: 1min


In [9]:
%%time
for col in text_cols:
  print("Processing column: {}".format(col))
  print("Processing train data... ")
  train_df[col] = train_df[col].apply(word_tokenize)
  print("Processing test data... ")
  test_df[col] = test_df[col].apply(word_tokenize)

Processing column: text
Processing train data... 
Processing test data... 
Processing column: title
Processing train data... 
Processing test data... 
Processing column: author
Processing train data... 
Processing test data... 
CPU times: user 1min 2s, sys: 644 ms, total: 1min 2s
Wall time: 1min 2s


In [ ]:
%%time
for col in text_cols:
  print("Processing column: {}".format(col))
  print("Processing train data... ")
  train_df[col] = train_df[col].apply(lambda x: [each_word for each_word in x if each_word not in stopwords])
  print("Processing test data... ")
  test_df[col] = test_df[col].apply(lambda x: [each_word for each_word in x if each_word not in stopwords])

Processing column: text
Processing train data... 


In [ ]:
print("Processing train data... ")
train_df['all_info'] = train_df['text'] + train_df['title'] + train_df['author'] 
train_df['all_info'] = train_df['all_info'].apply(lambda x: " ".join(x))
print("Processing test data... ")
test_df['all_info'] = test_df['text'] + test_df['title'] + test_df['author'] 
test_df['all_info'] = test_df['all_info'].apply(lambda x: " ".join(x))

In [ ]:
all_text_train = train_df["all_info"].astype(str).tolist()
all_text_test = test_df["all_info"].astype(str).tolist()

In [ ]:
target = train_df['label'].values

In [ ]:
tokenizer = Tokenizer(oov_token = "<OOV>", num_words=6000)
tokenizer.fit_on_texts(all_text_train)
word_index = tokenizer.word_index
len(word_index)

In [ ]:
sequences_train = tokenizer.texts_to_sequences(all_text_train)
padded_train = pad_sequences(sequences_train, padding = 'post', maxlen=6000)

sequences_test = tokenizer.texts_to_sequences(all_text_test)
padded_test = pad_sequences(sequences_test, padding = 'post', maxlen=6000)

In [ ]:
def get_model():
  model=Sequential()
  model.add(Embedding(6000, 300 ,input_length=6000))
  model.add(Dropout(0.3))
  model.add(LSTM(200))
  model.add(Dropout(0.3))
  model.add(Dense(64,activation='relu'))
  model.add(Dropout(0.3))
  model.add(Dense(1,activation='sigmoid'))
  return model

model = get_model()
model.summary()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(padded_train, target, test_size=0.2)

In [ ]:
callbacks=[
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=15, 
                                  verbose=1, mode="min", restore_best_weights=True),
    keras.callbacks.ModelCheckpoint(filepath="best_model.hdf5", verbose=1, save_best_only=True)
]

In [ ]:
model = get_model()
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
%%time
history = model.fit(X_train, 
                    y_train, 
                    epochs=20,
                    batch_size=64, 
                    validation_data=(X_test, y_test), 
                    callbacks=callbacks)

In [ ]:
metric_toplot = "loss"
plt.plot(history.epoch, history.history[metric_toplot], ".:", label="loss")
plt.plot(history.epoch, history.history["val_"+metric_toplot], ".:", label="val_loss")
plt.legend()
plt.show()

In [ ]:
model = keras.models.load_model('best_model.hdf5')

In [ ]:
y_pred = model.predict_classes(X_test)

In [ ]:
y_test.shape, y_pred.shape

In [ ]:
precision_score(y_test, y_pred)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
recall_score(y_test, y_pred)

In [ ]:
test_df

In [ ]:
y_pred_test = model.predict_classes(padded_test)

In [ ]:
submit_lstm_predictions_df = pd.DataFrame()
submit_lstm_predictions_df['id'] = test_df['id']
submit_lstm_predictions_df['label'] = y_pred_test.astype(int)

In [ ]:
submit_lstm_predictions_df

In [ ]:
submit_lstm_predictions_df['label'].value_counts()

In [ ]:
sns.countplot(submit_lstm_predictions_df['label'])
plt.title("Count of Genuine and Fake News of the Test Data")
plt.show()

In [ ]:
submit_predictions_df.to_csv('submit.csv',index=False)